This notebook introduces the h2ohyperopt module for Python. Here we begin with using single model optimizers and introduce the functions available to a user.

In [1]:
#Imports
import h2o
import h2ohyperopt

In [2]:
#Initializing h2o
h2o.init()

H2O cluster uptime:,3 hours 26 minutes 33 seconds 999 milliseconds
H2O cluster version:,3.8.3.3
H2O cluster name:,H2O_started_from_python_abhishek_klm767
H2O cluster total nodes:,1
H2O cluster total free memory:,1020.61 MB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


### Data Processing
The test dataset used for demonstrating the capabilities of H2OHyperopt is the titanic dataset. The function ```data()``` is used to preprocess the dataset.

In [3]:
def data():
    """
    Function to process the example titanic dataset.
    Train-Valid-Test split is 60%, 20% and 20% respectively.
    Output
    ---------------------
    trainFr: Training H2OFrame.
    testFr: Test H2OFrame.
    validFr: Validation H2OFrame.
    predictors: List of predictor columns for the Training frame.
    response: String defining the response column for Training frame.
    """
    titanic_df = h2o.import_file(path="https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")

    # Basic preprocessing
    # columns_to_be_used - List of columns which are used in the training/test
    # data
    # columns_to_factorize - List of columns with categorical variables
    columns_to_be_used = ['pclass', 'age', 'sex', 'sibsp', 'parch', 'ticket',
                          'embarked', 'fare', 'survived']
    columns_to_factorize = ['pclass', 'sex', 'sibsp', 'embarked', 'survived']
    # Factorizing the columns in the columns_to_factorize list
    for col in columns_to_factorize:
        titanic_df[col] = titanic_df[col].asfactor()
    # Selecting only the columns we need
    titanic_frame = titanic_df[columns_to_be_used]
    trainFr, testFr, validFr = titanic_frame.split_frame([0.6, 0.2],
                                                         seed=1234)
    predictors = trainFr.names[:]
    # Removing the response column from the list of predictors
    predictors.remove('survived')
    response = 'survived'
    return trainFr, testFr, validFr, predictors, response

In [4]:
trainFr, testFr, validFr, predictors, response = data()


Parse Progress: [##################################################] 100%


### Single Model Type Based Optimization

Let us demonstrate the GBMOptimizer and extend the example to DLEOptimizer. Since the problem is a binary classification problem, we specify the metric to AUC.

In [5]:
model_gbm = h2ohyperopt.GBMOptimizer(metric='auc')

Now we need to fix the search space for the models. H2ohyperopt has built in default search spaces.

In [11]:
#To find the searchable parameters
model_gbm.list_all_parameters()

['learn_rate',
 'fold_column',
 'response_column',
 'stopping_metric',
 'score_tree_interval',
 'sample_rate_per_class',
 'seed',
 'weights_column',
 'nfolds',
 'max_after_balance_size',
 'offset_column',
 'checkpoint',
 'stopping_tolerance',
 'training_frame',
 'col_sample_rate_per_tree',
 'max_runtime_secs',
 'max_abs_leafnode_pred',
 'quantile_alpha',
 'balance_classes',
 'learn_rate_annealing',
 'sample_rate',
 'validation_frame',
 'max_depth',
 'model_id',
 'build_tree_one_node',
 'ntrees',
 'min_split_improvement',
 'ignored_columns',
 'tweedie_power',
 'min_rows',
 'score_each_iteration',
 'max_confusion_matrix_size',
 'col_sample_rate_change_per_level',
 'nbins',
 'histogram_type',
 'col_sample_rate',
 'fold_assignment',
 'keep_cross_validation_predictions',
 'stopping_rounds',
 'nbins_top_level',
 'max_hit_ratio_k',
 'nbins_cats',
 'r2_stopping',
 'distribution',
 'class_sampling_factors',
 'ignore_const_cols',
 'keep_cross_validation_fold_assignment']

In [6]:
# To use the default search space
# model_gbm.select_optimization_parameters("Default")
# To use a combination of Default parameters and the customized parameters.
model_gbm.select_optimization_parameters({'col_sample_rate': 'Default',
                                          'ntrees': 200,
                                          'learn_rate': ('uniform',(0.05, 0.2)),
                                          'nfolds': 7})

Starting the evaluation of models. Let the maximum number of models to be evaluated be 100.

In [9]:
model_gbm.start_optimization(num_evals=10, trainingFr=trainFr,
                         validationFr=validFr, response=response,
                         predictors=predictors)


gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%


To get the best model from the optimization process,

In [10]:
model_gbm.best_model 

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1470344234567_196158

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,52388.0,5.0,5.0,5.0,7.0,28.0,15.88




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0550450629152
R^2: 0.767723341973
LogLoss: 0.20165508211
Mean Per-Class Error: 0.0601717309733
AUC: 0.984574730023
Gini: 0.969149460046

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.398409540671: 


,0,1,Error,Rate
0,457.0,22.0,0.0459,(22.0/479.0)
1,24.0,277.0,0.0797,(24.0/301.0)
Total,481.0,299.0,0.059,(46.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid
accuracy,0.8305758,0.0392350,0.7008547,0.8793104,0.8235294,0.8571429,0.8655462,0.8407080,0.8469388
auc,0.8587753,0.0363450,0.7539451,0.9253539,0.8666667,0.9020824,0.8629129,0.8282051,0.8722610
err,0.1694242,0.0392350,0.2991453,0.1206896,0.1764706,0.1428571,0.1344538,0.1592920,0.1530612
err_count,19.0,4.8989797,35.0,14.0,21.0,14.0,16.0,18.0,15.0
f0point5,0.7882997,0.0369742,0.6948640,0.8505155,0.8043478,0.7960199,0.8549223,0.754717,0.7627119
f1,0.7805185,0.0269414,0.7244095,0.825,0.7789474,0.8205128,0.8048781,0.7272728,0.7826087
f2,0.7749867,0.0303856,0.7565789,0.8009709,0.7551020,0.8465608,0.7603686,0.7017544,0.8035714
lift_top_group,2.2352314,0.6909254,1.9830508,2.7619047,2.38,2.6486487,2.6444445,3.2285714,0.0
logloss,0.4784969,0.0950435,0.7790785,0.3166093,0.4518574,0.4078223,0.4180915,0.4743263,0.5016928
max_per_class_error,0.2519868,0.0518419,0.3793103,0.2142857,0.26,0.1475410,0.2666667,0.3142857,0.1818182



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-08-04 17:42:46,4.940 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-08-04 17:42:46,4.944 sec,1.0,0.2224041,0.6363320,0.8791433,2.5913621,0.1961538
,2016-08-04 17:42:46,4.948 sec,2.0,0.2113311,0.6132219,0.8856942,2.5913621,0.1807692
,2016-08-04 17:42:46,4.952 sec,3.0,0.1999806,0.5896006,0.8896615,2.5913621,0.1705128
,2016-08-04 17:42:46,4.957 sec,4.0,0.1900763,0.5687925,0.8995901,2.5913621,0.1769231
---,---,---,---,---,---,---,---,---
,2016-08-04 17:42:50,8.767 sec,141.0,0.0667204,0.2369373,0.9741918,2.5913621,0.0807692
,2016-08-04 17:42:50,8.812 sec,142.0,0.0664990,0.2362849,0.9742785,2.5913621,0.0807692
,2016-08-04 17:42:50,8.860 sec,143.0,0.0661484,0.2353313,0.9749235,2.5913621,0.0794872
,2016-08-04 17:42:50,8.906 sec,144.0,0.0656871,0.2340608,0.9755200,2.5913621,0.0794872



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sex,390.4053955,1.0,0.3740837
fare,173.6464081,0.4447849,0.1663868
pclass,144.3788757,0.3698178,0.1383428
age,133.7865448,0.3426862,0.1281933
ticket,105.5445328,0.2703460,0.1011320
sibsp,36.0375671,0.0923081,0.0345309
embarked,36.0124931,0.0922438,0.0345069
parch,23.8192520,0.0610116,0.0228234


In [12]:
# To get the best model parameters
model_gbm.best_model_parameters()

{'col_sample_rate': 0.5384432216102001,
 'learn_rate': 0.06577581908265129,
 'metric': 'auc',
 'nfolds': 7,
 'ntrees': 200}

In [13]:
# To ensemble top 5 GBM models
model_gbm.best_model_ensemble(num_models=5)


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%
Model Trained


In [14]:
#Test score of the ensembler model
model_gbm.score_ensemble(testFr)


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%
  predict    predict0    predict1    predict2    predict3    survived
---------  ----------  ----------  ----------  ----------  ----------
        1           1           1           1           1           1
        1           1           1           1           1           0
        1           1           1           1           1           0
        0           0           0           0           1           1
        1           1           1           1           1           1
        0           1           0           0           0           1
        0           

0.7439278670047901

In [15]:
model_gbm.ensemble_model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1470344234567_211599

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,18750.0,0.0,3.0,0.865,1.0,5.0,2.265




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0146422803035
R^2: 0.93821317018
LogLoss: 0.0694820194116
Mean Per-Class Error: 0.0133341193933
AUC: 0.988639122202
Gini: 0.977278244405

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.210152914365: 


,0,1,Error,Rate
0,471.0,8.0,0.0167,(8.0/479.0)
1,3.0,298.0,0.01,(3.0/301.0)
Total,474.0,306.0,0.0141,(11.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.9859191,0.0065572,0.9874214,0.9865772,0.9683544,0.993865,0.9933775
auc,0.9860642,0.0059489,0.9841824,0.9856741,0.9715799,0.9945238,0.9943609
err,0.0140809,0.0065572,0.0125786,0.0134228,0.0316456,0.0061350,0.0066225
err_count,2.2,1.0392305,2.0,2.0,5.0,1.0,1.0
f0point5,0.9766414,0.0098346,0.9766764,0.9833333,0.9498208,0.9874608,0.9859155
f1,0.9813718,0.0096351,0.9852941,0.9833333,0.954955,0.992126,0.9911504
f2,0.9861640,0.0098333,0.9940653,0.9833333,0.9601449,0.9968355,0.9964413
lift_top_group,2.2727728,0.2702420,1.5820895,2.4397662,2.1545455,2.5419104,2.6455526
logloss,0.0830252,0.0297414,0.0862512,0.0822879,0.1585925,0.0377614,0.0502331
max_per_class_error,0.0190592,0.0068354,0.0217391,0.0166667,0.0363636,0.01,0.0105263



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-08-04 17:49:36,4.331 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-08-04 17:49:36,4.335 sec,1.0,0.1947537,0.5783735,0.9862012,2.5356339,0.0166667
,2016-08-04 17:49:36,4.338 sec,2.0,0.1609672,0.5073369,0.9862012,2.5356339,0.0166667
,2016-08-04 17:49:36,4.341 sec,3.0,0.1337744,0.4488793,0.9862012,2.5356339,0.0166667
,2016-08-04 17:49:36,4.345 sec,4.0,0.1118067,0.3999529,0.9862012,2.5356339,0.0166667
---,---,---,---,---,---,---,---,---
,2016-08-04 17:49:39,7.743 sec,196.0,0.0146423,0.0694820,0.9886391,2.4679639,0.0141026
,2016-08-04 17:49:39,7.769 sec,197.0,0.0146423,0.0694820,0.9886391,2.4679639,0.0141026
,2016-08-04 17:49:39,7.796 sec,198.0,0.0146423,0.0694820,0.9886391,2.4679639,0.0141026
,2016-08-04 17:49:39,7.824 sec,199.0,0.0146423,0.0694820,0.9886391,2.4679639,0.0141026



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
predict3,667.1802368,1.0,0.7181271
predict2,259.3450623,0.3887181,0.2791490
predict1,1.3190031,0.0019770,0.0014197
predict0,0.6235051,0.0009345,0.0006711
predict,0.5881464,0.0008815,0.0006331
